In [1]:
import torch
import numpy as np
from torch.utils.data import TensorDataset
import pandas as pd

In [2]:
data = pd.read_csv('large_field_preprocessed_data.csv')

In [3]:
dropColumns = [
    "SERIAL",
    "PERNUM",
    "HHWT",
    "CLUSTER",
    "STRATA",
    "PERWT",
    "YRMARR",
    "YRNATUR",
    "RACNUM",
    "index",
]

data = data.drop(columns=dropColumns)

for column in data.columns:
	data[column].fillna(False, inplace=True)
 
# data = data.drop(data[(data['INCWAGE_CPIU_2010'] == 0)].index)

In [4]:
data['INCWAGE_CPIU_2010'] = data['INCWAGE_CPIU_2010'].astype(np.float32)

for column in data.columns:
	if column == "INCWAGE_CPIU_2010":
		continue
	data[column] = data[column].astype(np.float32)

In [5]:
xTensor = torch.tensor(data.drop(columns=['INCWAGE_CPIU_2010']).values, dtype=torch.float32)
yTensor = torch.tensor(data['INCWAGE_CPIU_2010'].values, dtype=torch.float32)

In [12]:
dataset = TensorDataset(xTensor, yTensor)
trainSize = int(0.9 * len(dataset))
testSize = len(dataset) - trainSize
trainSet, testSet = torch.utils.data.random_split(dataset, [trainSize, testSize])


torch.save(trainSet, 'dataset/trainSet.pt')
torch.save(testSet, 'dataset/testSet.pt')